##Instructions for Creating a PostgreSQL Database for San Francisco Neighborhood Blocks

1. Brew install <b>PostGIS</b> :

 `brew install postgis`
  
  This library allows you to load .shp files and then export GeoJSON strings.


2. Whever you initialize a database, you have to give it post gis capabilities:
 ```sql
CREATE DATABASE <dbname>;
\q
psql <dbname>
CREATE EXTENSION POSTGIS 
```

3. Place shape files downloaded from the cenusus into PostgreSQL  This will create a table if one doesn't exist already. 
 
    `ogr2ogr -f "PostgreSQL" PG:"dbname= <dbname> user=<your psql username>" "<path to shapefile>/<shape file>" -nlt PROMOTE_TO_MULTI -nln <table name> -append`

    `ogr2ogr -f "PostgreSQL" PG:"dbname= zipfiantwitter user=clwilloughby" "/Users/clwilloughby/Documents/root/repos/mapping/data/tl_2014_06_tabblock10/" -nlt PROMOTE_TO_MULTI -nln sf_neighb -append`
    
    I had difficulties with instillation, and had to uninstall brew, then reinstall with:  
  
    ` brew install gdal --with-postgresql`


 /Users/clwilloughby/Documents/root/repos/mapping/data/tl_2014_06_tabblock10/tl_2014_06_tabblock10.shp

In [ ]:
database: zipfiantwitter
table with data: tweeted 

###Useful Commands:
```SQL
float ST_Area(geometry g1);
float ST_Area(geography geog, boolean use_spheroid=true);
```
Returns the area of the geometry if it is a polygon or multi-polygon.

###Get geometry columns in SQL formatted to work with your own lat/long data:

```sql
SELECT AddGeometryColumn('tweeted','geom',4326,'POINT',2);
UPDATE tweeted SET geom = ST_SetSRID(ST_MakePoint(lons, lats), 4326);
SELECT UpdateGeometrySRID('sf_neighb', 'wkb_geometry', 4326);
```

Now have a new column <b>geom</b> in tweeted, which has lats and lons. This is a unique geometry object based on the lats and lons values. 


Join only places where points are within the other point
Returns everything from points, and the id from the polys table


```SQL
 
SELECT points.*, polys.geoid10
INTO tweeted_neighb 
           FROM sf_neighb polys
           JOIN tweeted points 
               ON ST_Within(points.geom,polys.wkb_geometry)
```

This worked, so now I want unique geoid10 for each area
Now I have a new table with a <b>geoid10</b> column
Called <b>tweeted_neighb</b>

```SQL 
SELECT points.*, polys.geoid10
 INTO tweeted_neighb 
            FROM sf_neighb polys
            JOIN tweeted points 
                ON ST_Within(points.geom,polys.wkb_geometry);
```



